<a href="https://colab.research.google.com/github/BengIslam7/TinyML-SensorsValuesPrediction/blob/main/Anomaly_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip -q '/content/gdrive/MyDrive/iot_dataset.zip'

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2

In [4]:
data = pd.read_csv('/content/DATA-large.CSV')

In [433]:
data.head()

,time,temperature,humidity,pressure,lux
0,2022/3/12 22:32:2,24.2,32.7,98490.45,10.00
1,2022/3/12 22:32:6,24.0,33.9,98486.98,12.08
2,2022/3/12 22:32:11,24.1,33.9,98488.27,22.92
3,2022/3/12 22:32:16,24.1,33.8,98489.58,22.92
4,2022/3/12 22:32:21,24.1,35.0,98489.49,22.92


In [6]:
temp = data['temperature'].values
real_temp = data['temperature'].values

In [7]:
temp

array([24.2, 24. , 24.1, ..., 23.3, 23.3, 23.3])

In [8]:
temp.shape

(693220,)

In [551]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
temp = scaler.fit_transform(temp.reshape(-1,1))

In [552]:
temp

array([[0.47272727],
       [0.45454545],
       [0.46363636],
       ...,
       [0.39090909],
       [0.39090909],
       [0.39090909]])

In [553]:
temp.shape

(693220, 1)

In [554]:
X, y = [],[]
for i in range(0,temp.shape[0] - 500,5):
  X.append(temp[i:i+10])
  y.append(temp[i+10])
X=np.array(X)
y=np.array(y)
X = X.reshape(X.shape[0], -1)
print(X.shape, y.shape)

(138544, 10) (138544, 1)


In [555]:
# Parameters
n_estimators = 100  # Number of trees
contamination = 0.01  # Expected proportion of anomalies
sample_size = 256  # Number of samples used to train each tree

In [556]:
from sklearn.ensemble import IsolationForest
model = IsolationForest(n_estimators=n_estimators,
                            contamination=contamination,
                            max_samples=sample_size,
                            random_state=42)
model.fit(X)


IsolationForest(contamination=0.01, max_samples=256, random_state=42)

In [557]:
windows = []
for i in range(692720,693220-10,5):
  window = temp[i:i+10].reshape(1, -1)
  windows.append(window)

In [558]:

windows = np.array(windows)

In [559]:
windows.shape

(98, 1, 10)

In [560]:
windows=windows.reshape(windows.shape[0],windows.shape[2])

In [561]:
windows

array([[0.37272727, 0.37272727, 0.37272727, 0.37272727, 0.37272727,
        0.37272727, 0.38181818, 0.37272727, 0.37272727, 0.37272727],
       [0.37272727, 0.38181818, 0.37272727, 0.37272727, 0.37272727,
        0.37272727, 0.37272727, 0.37272727, 0.38181818, 0.37272727],
       [0.37272727, 0.37272727, 0.37272727, 0.38181818, 0.37272727,
        0.37272727, 0.37272727, 0.37272727, 0.37272727, 0.37272727],
       [0.37272727, 0.37272727, 0.37272727, 0.37272727, 0.37272727,
        0.38181818, 0.37272727, 0.37272727, 0.37272727, 0.37272727],
       [0.38181818, 0.37272727, 0.37272727, 0.37272727, 0.37272727,
        0.37272727, 0.37272727, 0.37272727, 0.37272727, 0.37272727],
       [0.37272727, 0.37272727, 0.37272727, 0.37272727, 0.37272727,
        0.37272727, 0.37272727, 0.37272727, 0.37272727, 0.37272727],
       [0.37272727, 0.37272727, 0.37272727, 0.37272727, 0.37272727,
        0.37272727, 0.37272727, 0.37272727, 0.37272727, 0.37272727],
       [0.37272727, 0.37272727, 0.3727272

In [562]:
scores = model.decision_function(windows)

In [563]:
scores

array([0.21959275, 0.2156135 , 0.21341918, 0.21775449, 0.21945304,
       0.22387361, 0.22387361, 0.21775449, 0.21945304, 0.21662248,
       0.19618029, 0.21192264, 0.20462917, 0.2156135 , 0.20794495,
       0.20456631, 0.21222759, 0.20047412, 0.21955324, 0.20088943,
       0.20928259, 0.20846693, 0.2142756 , 0.20449154, 0.21244944,
       0.22667921, 0.23043547, 0.23043547, 0.23043547, 0.23043547,
       0.23043547, 0.23043547, 0.23043547, 0.23043547, 0.23043547,
       0.23043547, 0.23043547, 0.23043547, 0.23043547, 0.23043547,
       0.23043547, 0.23043547, 0.23043547, 0.23043547, 0.23043547,
       0.23043547, 0.23043547, 0.23043547, 0.23043547, 0.23043547,
       0.23043547, 0.23043547, 0.23043547, 0.22505056, 0.220475  ,
       0.23043547, 0.2238674 , 0.21480451, 0.21940621, 0.23043547,
       0.22505056, 0.21245546, 0.20804903, 0.20733244, 0.21098837,
       0.21086577, 0.21324726, 0.20953733, 0.20815887, 0.21451457,
       0.21181246, 0.20534662, 0.21223451, 0.22122752, 0.21056

In [564]:
len(scores)

98

In [565]:
anomaly = model.predict(windows)

In [566]:
anomaly

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [567]:
test = [26.0 , 27.0 , 28.2 , 29.3 , 30.5, 31.8 , 32.7 , 33.8, 34.9 , 0.9]

In [568]:
test = np.array(test)

In [569]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
test = scaler.fit_transform(test.reshape(-1,1))

In [570]:
test.shape

(10, 1)

In [571]:
test = test.reshape(1,-1)

In [572]:
test

array([[0.73823529, 0.76764706, 0.80294118, 0.83529412, 0.87058824,
        0.90882353, 0.93529412, 0.96764706, 1.        , 0.        ]])

In [573]:
test.shape

(1, 10)

In [574]:
scores = model.decision_function(test)

In [575]:
scores

array([-0.06502847])

In [576]:
anomaly = model.predict(test)

In [577]:
threshold = np.percentile(scores, 5)  # or 1–5%
anomaly = scores[0] < threshold

In [578]:
anomaly

np.False_